# 要約 
このJupyter Notebookは、LMSYS - Chatbot Arenaコンペティションにおけるゼロショット予測に関するもので、特にLLama3モデルを使用して、ユーザーのプロンプトに対して応答モデルの適切さを予測することに焦点を当てています。Notebookは大きく分けて、ライブラリのインポート、データの読み込み、モデルの準備、予測の実行、結果の保存と評価の5つのセクションで構成されています。

### 問題の特定:
このNotebookは、ユーザーが選好する応答を予測するタスクに取り組んでいます。具体的には、Chatbot Arenaから得られたデータを用いて、与えられたプロンプトに対して選択肢として与えられる応答A、B、そのいずれかが適切か、または両方が同等か（引き分け）を予測します。

### 手法とライブラリ:
- **手法**: プロンプト工夫によるゼロショット学習を用いて、モデルに対して強化されたプロンプトを提供し、最も可能性の高い応答を予測します。モデルの出力から得た結果を元に、正規化された確率を計算することによって対数損失を評価します。

- **使用ライブラリ**:
  - `transformers`: Hugging FaceのTransformersライブラリを使用し、LLama3モデルとそのトークナイザーを導入しています。
  - `torch`: PyTorchを用いて、モデルのトレーニングおよび推論を行います。
  - `pandas`と`numpy`: データフレームの作成や数値演算のために使用されます。
  - `sklearn.metrics`: モデルの精度を評価するために対数損失を算出するために利用されています。

Notebookは、全体を通してプロセスの各ステップを明確に示し、最終的には予測結果をCSVファイルとして保存し、評価指標を表示しています。また、次のステップとして、さらなるモデルの改良やプロンプトの強化、教師ありファインチューニング（SFT）、アンサンブル手法の検討を提案しています。

---


# 用語概説 
以下は、Jupyter Notebook の内容に関連する専門用語の解説です。特に初心者がつまずきやすいポイントや、コンペティション特有のドメイン知識に焦点を当てました。

1. **ゼロショット予測 (Zero-shot prediction)**:
   ゼロショット予測とは、モデルが訓練されていないタスクに対しても、適切に応答や予測を行う能力のことを指します。このノートブックでは、特定の分類モデルを訓練するのではなく、入力に対して直接プロンプトを与え、最も適切な応答を選ぶことを試みています。

2. **トークン (Token)**:
   大きなテキストを処理可能な小さな単位に分割したものです。トークンには単語や部分語、さらには記号も含まれることがあります。自然言語処理では、モデルに入力するためにテキストをトークナイズする必要があります。

3. **因果言語モデル (Causal Language Model)**:
   前の単語から現在の単語を順に予測する構造を持つ言語モデルです。このノートブックでは、因果関係を考慮した生成タスクを実施しています。

4. **トークナイザー (Tokenizer)**:
   テキストをトークンに変換する処理を行うツールです。トークン化されることで、モデルがテキストデータを理解しやすくなります。また、トークナイザーは、逆にトークンをテキストに戻す役割も果たします。

5. **ソフトマックス (Softmax)**:
   複数のクラスに対する確率を示すための関数です。モデルの出力を操作して、数値を確率に変換し、合計が1になるように正規化します。

6. **勾配計算 (Gradient Calculation)**:
   モデルのパラメータを更新するために用いる手法で、損失関数に対するパラメータの変化を測定します。通常、訓練時に用いられますが、このコードでは予測時に勾配計算を無効にしています。

7. **ログ損失 (Log Loss)**:
   確率の予測がどれだけ正確であるかを測るための指標です。対数関数を使って、実際のラベルと予測されたクラス間の乖離を評価します。小さい値がより優れた予測を意味します。

8. **デバイスマップ (Device Map)**:
   モデルのパラメータをどの計算デバイス（CPUやGPU）に配置するかを決定する設定です。`device_map='auto'`によって、PyTorchは自動的に最適なデバイスを選択します。

9. **プロンプト (Prompt)**:
   モデルに対して応答を求めるための入力文です。プロンプトに基づいてモデルは様々な応答を生成します。例では、応答の優劣を判断するためのプロンプトが設定されています。

10. **トークンID (Token ID)**:
    トークンに対応する整数の識別子です。モデルがトークンを処理する際には、これらの数値を扱います。

これらの用語について理解しておくことで、該当するコードの振る舞いや各処理の目的をより深く理解できるようになるでしょう。

---


<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# LMSYS ZeroShot Prediction

This code uses LLama3 to make zero-shot predictions.

Instead of learning a classification header, it uses a carefully crafted prompt to predict which token among **A**, **B**, or **tie** is most likely to follow `###Answer:`.









</div>
<div class="column-right">

# 日本語訳

# LMSYS ゼロショット予測

このコードはLLama3を使用してゼロショット予測を行います。

分類ヘッダーを学習する代わりに、巧妙に設計されたプロンプトを使用して、`###Answer:`の後に最も可能性が高いトークンである**A**、**B**、または**tie**のいずれかを予測します。

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import json
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import pickle
import random
import os
import sys

import transformers
from transformers import AdamW
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_cosine_schedule_with_warmup

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

import sklearn.metrics
from sklearn.metrics import accuracy_score

os.environ["TOKENIZERS_PARALLELISM"] = "true"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
```

</div>
<div class="column-right">

# 日本語訳

```python
import json  # JSON形式のデータを処理するためのライブラリをインポート
import pandas as pd  # データ解析のためのライブラリをインポート
import numpy as np  # 数値計算のためのライブラリをインポート
from tqdm.auto import tqdm  # プログレスバーを表示するためのライブラリをインポート
import pickle  # オブジェクトの保存と読み込みを行うためのライブラリをインポート
import random  # ランダムな数値生成のためのライブラリをインポート
import os  # オペレーティングシステムとのインタラクションのためのライブラリをインポート
import sys  # Pythonインタプリタとのインタラクションのためのライブラリをインポート

import transformers  # トランスフォーマモデルのライブラリをインポート
from transformers import AdamW  # AdamWオプティマイザをインポート
from transformers import AutoTokenizer, AutoModel, AutoConfig  # 自動トークナイザー、モデル、設定をインポート
from transformers import get_cosine_schedule_with_warmup  # コサインスケジュールをインポート

import torch  # PyTorchライブラリをインポート
import torch.nn as nn  # ニューラルネットワーク機能のモジュールをインポート
from torch.utils.data import DataLoader, Dataset  # データローダーとデータセットの入出力を行うためのモジュールをインポート

import sklearn.metrics  # sklearnのメトリクスライブラリをインポート
from sklearn.metrics import accuracy_score  # 精度スコアを計算するための関数をインポート

# トークナイザの並列処理を有効にするための環境変数を設定
os.environ["TOKENIZERS_PARALLELISM"] = "true"
# 使用可能な場合はGPUを、そうでなければCPUをデバイスとして設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 使用するデバイスを出力する
print(device)  # 現在のデバイス（CPUまたはGPU）を表示
```

</div>
</details>

In [ ]:
import json  # JSON形式のデータを処理するためのライブラリをインポート
import pandas as pd  # データ解析のためのライブラリをインポート
import numpy as np  # 数値計算のためのライブラリをインポート
from tqdm.auto import tqdm  # プログレスバーを表示するためのライブラリをインポート
import pickle  # オブジェクトの保存と読み込みを行うためのライブラリをインポート
import random  # ランダムな数値生成のためのライブラリをインポート
import os  # オペレーティングシステムとのインタラクションのためのライブラリをインポート
import sys  # Pythonインタプリタとのインタラクションのためのライブラリをインポート

import transformers  # トランスフォーマモデルのライブラリをインポート
from transformers import AdamW  # AdamWオプティマイザをインポート
from transformers import AutoTokenizer, AutoModel, AutoConfig  # 自動トークナイザー、モデル、設定をインポート
from transformers import get_cosine_schedule_with_warmup  # コサインスケジュールをインポート

import torch  # PyTorchライブラリをインポート
import torch.nn as nn  # ニューラルネットワーク機能のモジュールをインポート
from torch.utils.data import DataLoader, Dataset  # データローダーとデータセットの入出力を行うためのモジュールをインポート

import sklearn.metrics  # sklearnのメトリクスライブラリをインポート
from sklearn.metrics import accuracy_score  # 精度スコアを計算するための関数をインポート

# トークナイザの並列処理を有効にするための環境変数を設定
os.environ["TOKENIZERS_PARALLELISM"] = "true"
# 使用可能な場合はGPUを、そうでなければCPUをデバイスとして設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 使用するデバイスを出力する
print(device)  # 現在のデバイス（CPUまたはGPU）を表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
class CFG:
    INPUT_DIR = "/kaggle/input/lmsys-chatbot-arena/"
    OUTPUT = "/kaggle/working"
    MODEL_ID = "/kaggle/input/llama-3/transformers/8b-hf/1"
    SEED = 42
    USE_TURN = 0
```

</div>
<div class="column-right">

# 日本語訳

```python
class CFG:
    # 入力データが保存されているディレクトリのパス
    INPUT_DIR = "/kaggle/input/lmsys-chatbot-arena/"
    # 出力データを保存するためのディレクトリのパス
    OUTPUT = "/kaggle/working"
    # 使用するモデルのIDが保存されているディレクトリのパス
    MODEL_ID = "/kaggle/input/llama-3/transformers/8b-hf/1"
    # 乱数の初期化に使用するシード値
    SEED = 42
    # ターンを使用するかどうかを示すフラグ。0は使用しないことを意味する
    USE_TURN = 0
```

</div>
</details>

In [ ]:
class CFG:
    # 入力データが保存されているディレクトリのパス
    INPUT_DIR = "/kaggle/input/lmsys-chatbot-arena/"
    # 出力データを保存するためのディレクトリのパス
    OUTPUT = "/kaggle/working"
    # 使用するモデルのIDが保存されているディレクトリのパス
    MODEL_ID = "/kaggle/input/llama-3/transformers/8b-hf/1"
    # 乱数の初期化に使用するシード値
    SEED = 42
    # ターンを使用するかどうかを示すフラグ。0は使用しないことを意味する
    USE_TURN = 0

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train_df = pd.read_csv(f"{CFG.INPUT_DIR}/train.csv")
train_df
```

</div>
<div class="column-right">

# 日本語訳

```python
# 訓練データをCSVファイルから読み込む
train_df = pd.read_csv(f"{CFG.INPUT_DIR}/train.csv")
# 読み込んだ訓練データを出力する
train_df  # 訓練データフレームの内容を表示
```

</div>
</details>

In [ ]:
# 訓練データをCSVファイルから読み込む
train_df = pd.read_csv(f"{CFG.INPUT_DIR}/train.csv")
# 読み込んだ訓練データを出力する
train_df  # 訓練データフレームの内容を表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    CFG.MODEL_ID,
    torch_dtype=torch.float16,
    device_map='auto',
)
```

</div>
<div class="column-right">

# 日本語訳

```python
from transformers import AutoModelForCausalLM, AutoTokenizer

# 事前に訓練された因果言語モデルをロードする
model = AutoModelForCausalLM.from_pretrained(
    CFG.MODEL_ID,  # モデルのIDを指定
    torch_dtype=torch.float16,  # モデルの重みを16ビット浮動小数点数で読み込む
    device_map='auto',  # 自動的にデバイスをマッピングする
)
```

</div>
</details>

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 事前に訓練された因果言語モデルをロードする
model = AutoModelForCausalLM.from_pretrained(
    CFG.MODEL_ID,  # モデルのIDを指定
    torch_dtype=torch.float16,  # モデルの重みを16ビット浮動小数点数で読み込む
    device_map='auto',  # 自動的にデバイスをマッピングする
)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
tokenizer = AutoTokenizer.from_pretrained(
    CFG.MODEL_ID,
    use_fast=False,
    trust_remote_code=True,
    padding_side="left",
    pad_token="<|endoftext|>"
)
model.config.pad_token_id = model.config.eos_token_id
```

</div>
<div class="column-right">

# 日本語訳

```python
# 事前に訓練されたトークナイザーをロードする
tokenizer = AutoTokenizer.from_pretrained(
    CFG.MODEL_ID,  # モデルのIDを指定
    use_fast=False,  # 高速トークナイザーを使用しない
    trust_remote_code=True,  # リモートのコードを信頼する
    padding_side="left",  # 左側にパディングを追加する設定
    pad_token="
```

</div>
</details>

In [ ]:
# 事前に訓練されたトークナイザーをロードする
tokenizer = AutoTokenizer.from_pretrained(
    CFG.MODEL_ID,  # モデルのIDを指定
    use_fast=False,  # 高速トークナイザーを使用しない
    trust_remote_code=True,  # リモートのコードを信頼する
    padding_side="left",  # 左側にパディングを追加する設定
    pad_token="

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
n_valid = len(train_df) // 5
sample_df = train_df.sample(n_valid, random_state=CFG.SEED)
sample_df.head()
```

</div>
<div class="column-right">

# 日本語訳

```python
# 訓練データの5分の1のサイズを計算する
n_valid = len(train_df) // 5
# 訓練データから無作為にサンプルを抽出する
sample_df = train_df.sample(n_valid, random_state=CFG.SEED)
# 抽出したサンプルデータの最初の5行を表示する
sample_df.head()  # サンプルデータフレームの先頭を表示
```

</div>
</details>

In [ ]:
# 訓練データの5分の1のサイズを計算する
n_valid = len(train_df) // 5
# 訓練データから無作為にサンプルを抽出する
sample_df = train_df.sample(n_valid, random_state=CFG.SEED)
# 抽出したサンプルデータの最初の5行を表示する
sample_df.head()  # サンプルデータフレームの先頭を表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
results = []
for _, row in tqdm(sample_df.iterrows(), total=len(sample_df)):
    prompt = json.loads(row["prompt"])
    response_a = json.loads(row["response_a"])
    response_b = json.loads(row["response_b"])
    
    
    p = prompt[CFG.USE_TURN]
    a = response_a[CFG.USE_TURN]
    b = response_b[CFG.USE_TURN]
    
    if a is None or b is None or len(p.split()) < 3:
        continue
        
    p = " ... ".join(["none" if i is None else i for i in prompt])
    a = " ... ".join(["none" if i is None else i for i in response_a])
    b = " ... ".join(["none" if i is None else i for i in response_b])
    # Use head and tail
    p = p[:128] + " ... " + p[-128:]
    a = a[:256] + " ... " + a[-256:]
    b = b[:256] + " ... " + b[-256:]
    
    text = f"""### Instruction
Which model's answer is appropriate for the prompt?　If both are appropriate, answer `tie`.

### Prompt
{p}

### A
{a}

### B
{b}

### Answer
"""

    toks = tokenizer(text)

    for k in toks.keys():
        toks[k] = torch.tensor(toks[k]).cuda()  

    with torch.no_grad():
        out = model(toks["input_ids"].unsqueeze(0))

    pred_token_id = tokenizer.encode("A") + tokenizer.encode("B") + tokenizer.encode("tie")
    pred = out.logits[0, -1, pred_token_id].cpu().softmax(0).numpy()
    
    d = row.to_dict()
    d["predict"] = pred
    results.append(d)
```

</div>
<div class="column-right">

# 日本語訳

```python
results = []  # 予測結果を格納するリストを初期化
# サンプルデータフレームの各行を無作為にイテレーションする
for _, row in tqdm(sample_df.iterrows(), total=len(sample_df)):
    # プロンプトと応答をJSON形式から読み込む
    prompt = json.loads(row["prompt"])  
    response_a = json.loads(row["response_a"])  
    response_b = json.loads(row["response_b"])  
    
    # プロンプトと応答から現在のターンを取得
    p = prompt[CFG.USE_TURN]
    a = response_a[CFG.USE_TURN]
    b = response_b[CFG.USE_TURN]
    
    # 応答がNoneの場合やプロンプトの単語数が3未満の場合はスキップする
    if a is None or b is None or len(p.split()) < 3:
        continue
        
    # 各リストの中のNoneを"none"で置き換え、1つの文字列に結合する
    p = " ... ".join(["none" if i is None else i for i in prompt])
    a = " ... ".join(["none" if i is None else i for i in response_a])
    b = " ... ".join(["none" if i is None else i for i in response_b])
    # プロンプトの先頭128文字と末尾128文字を保持し、残りを省略
    p = p[:128] + " ... " + p[-128:]
    a = a[:256] + " ... " + a[-256:]
    b = b[:256] + " ... " + b[-256:]
    
    # テキストをフォーマットする
    text = f"""### Instruction
Which model's answer is appropriate for the prompt?　If both are appropriate, answer `tie`.

### Prompt
{p}

### A
{a}

### B
{b}

### Answer
"""

    # トークナイザーを使ってテキストをトークン化する
    toks = tokenizer(text)

    # トークンをGPUのテンソルに変換する
    for k in toks.keys():
        toks[k] = torch.tensor(toks[k]).cuda()  

    # 勾配計算を無効にしてモデルを実行
    with torch.no_grad():
        out = model(toks["input_ids"].unsqueeze(0))

    # 予測トークンIDを生成する
    pred_token_id = tokenizer.encode("A") + tokenizer.encode("B") + tokenizer.encode("tie")
    # モデルの出力からソフトマックスを計算して予測を取得
    pred = out.logits[0, -1, pred_token_id].cpu().softmax(0).numpy()
    
    # 元の行を辞書形式に変換し、予測を追加
    d = row.to_dict()
    d["predict"] = pred
    results.append(d)  # 予測結果をリストに追加
```

</div>
</details>

In [ ]:
results = []  # 予測結果を格納するリストを初期化
# サンプルデータフレームの各行を無作為にイテレーションする
for _, row in tqdm(sample_df.iterrows(), total=len(sample_df)):
    # プロンプトと応答をJSON形式から読み込む
    prompt = json.loads(row["prompt"])  
    response_a = json.loads(row["response_a"])  
    response_b = json.loads(row["response_b"])  
    
    # プロンプトと応答から現在のターンを取得
    p = prompt[CFG.USE_TURN]
    a = response_a[CFG.USE_TURN]
    b = response_b[CFG.USE_TURN]
    
    # 応答がNoneの場合やプロンプトの単語数が3未満の場合はスキップする
    if a is None or b is None or len(p.split()) < 3:
        continue
        
    # 各リストの中のNoneを"none"で置き換え、1つの文字列に結合する
    p = " ... ".join(["none" if i is None else i for i in prompt])
    a = " ... ".join(["none" if i is None else i for i in response_a])
    b = " ... ".join(["none" if i is None else i for i in response_b])
    # プロンプトの先頭128文字と末尾128文字を保持し、残りを省略
    p = p[:128] + " ... " + p[-128:]
    a = a[:256] + " ... " + a[-256:]
    b = b[:256] + " ... " + b[-256:]
    
    # テキストをフォーマットする
    text = f"""### Instruction
Which model's answer is appropriate for the prompt?　If both are appropriate, answer `tie`.

### Prompt
{p}

### A
{a}

### B
{b}

### Answer
"""

    # トークナイザーを使ってテキストをトークン化する
    toks = tokenizer(text)

    # トークンをGPUのテンソルに変換する
    for k in toks.keys():
        toks[k] = torch.tensor(toks[k]).cuda()  

    # 勾配計算を無効にしてモデルを実行
    with torch.no_grad():
        out = model(toks["input_ids"].unsqueeze(0))

    # 予測トークンIDを生成する
    pred_token_id = tokenizer.encode("A") + tokenizer.encode("B") + tokenizer.encode("tie")
    # モデルの出力からソフトマックスを計算して予測を取得
    pred = out.logits[0, -1, pred_token_id].cpu().softmax(0).numpy()
    
    # 元の行を辞書形式に変換し、予測を追加
    d = row.to_dict()
    d["predict"] = pred
    results.append(d)  # 予測結果をリストに追加

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
results_df = pd.DataFrame(results)
results_df
```

</div>
<div class="column-right">

# 日本語訳

```python
# 予測結果をデータフレーム形式に変換する
results_df = pd.DataFrame(results)
# 生成した結果データフレームを表示する
results_df  # 予測結果のデータフレームの内容を表示
```

</div>
</details>

In [ ]:
# 予測結果をデータフレーム形式に変換する
results_df = pd.DataFrame(results)
# 生成した結果データフレームを表示する
results_df  # 予測結果のデータフレームの内容を表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
results_df.to_csv("result.csv", index=None)
```

</div>
<div class="column-right">

# 日本語訳

```python
# 予測結果をCSVファイルとして保存する
results_df.to_csv("result.csv", index=None)  # インデックスなしでCSVファイルに書き込む
```

</div>
</details>

In [ ]:
# 予測結果をCSVファイルとして保存する
results_df.to_csv("result.csv", index=None)  # インデックスなしでCSVファイルに書き込む

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!ls
```

</div>
<div class="column-right">

# 日本語訳

```python
# 現在の作業ディレクトリ内のファイルとフォルダの一覧を表示する
!ls
```

</div>
</details>

In [ ]:
# 現在の作業ディレクトリ内のファイルとフォルダの一覧を表示する
!ls

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
targets = results_df[["winner_model_a", "winner_model_b", "winner_tie"]].values

predicts = np.array(results_df["predict"].tolist())
```

</div>
<div class="column-right">

# 日本語訳

```python
# 結果データフレームからラベルとなるターゲットを取得し、NumPy配列に変換する
targets = results_df[["winner_model_a", "winner_model_b", "winner_tie"]].values

# 予測結果をリスト形式からNumPy配列に変換する
predicts = np.array(results_df["predict"].tolist())  # 予測結果を配列に変換
```

</div>
</details>

In [ ]:
# 結果データフレームからラベルとなるターゲットを取得し、NumPy配列に変換する
targets = results_df[["winner_model_a", "winner_model_b", "winner_tie"]].values

# 予測結果をリスト形式からNumPy配列に変換する
predicts = np.array(results_df["predict"].tolist())  # 予測結果を配列に変換

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
logloss = sklearn.metrics.log_loss(targets, predicts)
print(logloss)
```

</div>
<div class="column-right">

# 日本語訳

```python
# ターゲットと予測に基づいて対数損失を計算する
logloss = sklearn.metrics.log_loss(targets, predicts)
# 計算した対数損失を出力する
print(logloss)  # 対数損失の値を表示
```

</div>
</details>

In [ ]:
# ターゲットと予測に基づいて対数損失を計算する
logloss = sklearn.metrics.log_loss(targets, predicts)
# 計算した対数損失を出力する
print(logloss)  # 対数損失の値を表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!ls
```

</div>
<div class="column-right">

# 日本語訳

```python
# 現在の作業ディレクトリ内のファイルとフォルダの一覧を表示する
!ls
```

</div>
</details>

In [ ]:
# 現在の作業ディレクトリ内のファイルとフォルダの一覧を表示する
!ls

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

### Next steps:

- Try a larger model (with better benchmark accuracy)
- Refine the prompt.
- SFT (Supervised Fine-Tuning) on competition data
- Use ensemble 

</div>
<div class="column-right">

# 日本語訳

### 次のステップ:

- より大きなモデル（より良いベンチマーク精度を持つ）を試す
- プロンプトを改善する
- コンペティションデータに対してSFT（教師ありファインチューニング）を行う
- アンサンブルを使用する

</div>